# Lab 02: LSTM - solution

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = 'lstm_solution.ipynb'
    path_to_file = '/content/gdrive/My Drive/Labs/lecture08_labs/lab02_lstm' # Please adjust the path accordingly 
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Labs/lecture08_labs/lab02_lstm
/content/gdrive/.shortcut-targets-by-id/13MZjehRPW2F707gdYdXllrNodq3gbfiq/Labs/lecture08_labs/lab02_lstm


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 274 sec ( 4.56 min)<br> 
* Time for 1 epoch on GPU : 10.1 sec w/ GeForce GTX 1080 Ti <br>

In [ ]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [ ]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [ ]:
bs = 20

vocab_size = 10000


### Make a recurrent net class

In [ ]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.LSTM(      hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init, c_init ):
        
        g_seq                      =   self.layer1( word_seq )  
        h_seq , (h_final,c_final)  =   self.layer2( g_seq , (h_init,c_init) )      
        score_seq                  =   self.layer3( h_seq )
        
        return score_seq,  h_final , c_final


### Build the net. Choose the hidden size to be 300. How many parameters in total?

In [ ]:
hidden_size=300

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 300)
  (layer2): LSTM(300, 300)
  (layer3): Linear(in_features=300, out_features=10000, bias=True)
)
There are 6732400 (6.73 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [ ]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [ ]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 5
* sequence length = 35

In [ ]:
criterion = nn.CrossEntropyLoss()

my_lr = 5

seq_length = 35

### Function to evaluate the network on the test set

In [ ]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)
   
    h=h.to(device)
    c=c.to(device)
       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h, c  = net( minibatch_data, h , c)
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
        c=c.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 8 passes through the training set.

In [ ]:
start=time.time()

for epoch in range(8):
    
    # divide the learning rate by 3 except after the first epoch
    if epoch >= 2:
        my_lr = my_lr / 3
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h and c to be the zero vector
    h = torch.zeros(1, bs, hidden_size)
    c = torch.zeros(1, bs, hidden_size)

    # send them to the gpu    
    h=h.to(device)
    c=c.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        c=c.detach()
        # h=h.requires_grad_()
        # c=c.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h, c  = net( minibatch_data, h , c)
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 11.140082597732544 	 lr= 5 	 exp(loss)= 280.6248759369808
test: exp(loss) =  176.4419702724576

epoch= 1 	 time= 22.70071315765381 	 lr= 5 	 exp(loss)= 127.72422078871847
test: exp(loss) =  134.33645154480897

epoch= 2 	 time= 34.30411648750305 	 lr= 1.6666666666666667 	 exp(loss)= 81.69741853132007
test: exp(loss) =  113.9044108536774

epoch= 3 	 time= 45.74212694168091 	 lr= 0.5555555555555556 	 exp(loss)= 67.27318404174349
test: exp(loss) =  109.63826710104534

epoch= 4 	 time= 57.06359672546387 	 lr= 0.1851851851851852 	 exp(loss)= 62.35541427816925
test: exp(loss) =  108.09188562028926

epoch= 5 	 time= 68.30030012130737 	 lr= 0.0617283950617284 	 exp(loss)= 60.593397063198154
test: exp(loss) =  107.40498081555248

epoch= 6 	 time= 79.54876613616943 	 lr= 0.0205761316872428 	 exp(loss)= 59.95079200969282
test: exp(loss) =  106.9942965091796

epoch= 7 	 time= 90.84557461738586 	 lr= 0.006858710562414266 	 exp(loss)= 59.71837276758041
test: exp(loss) =  106.7643016

### Choose one sentence (taken from the test set)

In [ ]:
# SENTENCES FROM TEST SET

sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence2 = "some analysts expect oil prices to remain relatively"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# OR MAKE YOUR OWN SENTENCE. 
# NO CAPITAL LETTER ALLOWED. EACH WORD MUST BE IN THE ALLOWED VOCABULARY OF 10,000 WORDS

sentence6= "he was very"


# CHOOSE THE SENTENCE
mysentence = sentence1

### Convert the sentence into a vector, then send to GPU

In [ ]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[ 307],
        [1140],
        [ 334],
        [1486],
        [1786],
        [  64],
        [ 719],
        [ 377]], device='cuda:0')


### Set the initial hidden state to zero, then run the LSTM.

In [ ]:
h = torch.zeros(1, 1, hidden_size)
c = torch.zeros(1, 1, hidden_size)
h=h.to(device)
c=c.to(device)

scores , h, c = net(minibatch_data , h, c)

### Display the network prediction for the next word

In [ ]:
print(mysentence, '... \n')

utils.show_next_word(scores)

some analysts expect oil prices to remain relatively ... 

11.3%	 <unk>
6.2%	 high
4.4%	 strong
2.3%	 small
2.1%	 good
1.9%	 low
1.5%	 large
1.4%	 heavy
1.2%	 competitive
0.9%	 longer
0.9%	 major
0.9%	 volatile
0.9%	 thin
0.9%	 well
0.8%	 profitable
0.8%	 greater
0.8%	 short
0.7%	 bullish
0.6%	 flat
0.6%	 weak
0.6%	 bad
0.6%	 little
0.6%	 larger
0.6%	 hard
0.6%	 stable
0.5%	 fairly
0.5%	 cheap
0.5%	 positive
0.5%	 great
0.5%	 costly
